# Functional Programming

## Characteristics

- Function is a first-class object
- Recursion is used as a primary control structure
- There is a focus on list processing
  - Lists are often used with recursions on sublists as a substitude for loops
- The pure functional paradigm has no side effects
- Not statements, Use expression like function with arguments
- Describe what is to be computed instead of how to be computed
- There are higher-order functions that produce functions
- Functional Programming supposes data, not datum
  - processing several data is default

In [1]:
# Statement
x = 1 + 1
print(x)
# Expression
plus = lambda x, y: x+y
x = plus(1, 1)
print(x)

## Pure Function

#### The pure function returns the same result given the same arguments

### Benefits
- Formal provability
  - Can construct a mathematical proof easier
- Modularity
- Composability
- Ease of debugging and testing

## Avoiding Flow Control

- Typically, loop statements, **for** and **while**, or branch statements, **if** and **try** is used for flow control 
- But, It can have side effects, so each results can be different
- Also, It focus to how to be computed rather than what to be computed

### Encapsulation

- One simple way to focus **what** is to refactor code, and put the data structure into an isolated place, such as function or method

In [2]:
# Imperative
def condition(state_var):
    pass
def calculate_from(datum):
    pass
def modify(datum, stateVar):
    pass
def processing(thing):
    pass

collection = []
stateVar = None
dataset = [i for i in range(1, 10)]
for datum in dataset:
    if condition(stateVar):
        stateVar = calculate_from(datum)
        new = modify(datum, stateVar)
        collection.append(new)
        
# Encapsulation
def make_collection():
    collection = []
    stateVar = None
    dataset = [i for i in range(1, 10)]
    for datum in dataset:
        if condition(stateVar):
            stateVar = calculate_from(datum)
            new = modify(datum, stateVar)
            collection.append(new)
    return collection

collection = make_collection()

for thing in collection:
    processing(thing)

### Comprehension

- The comprehension is an expression that uses same keywords as loop and condition blocks, but inverts their order to focus on data
- The comprehension is optimistic, so faster but consum larger memory
- In Data Analysis, comprehension may not be used 'cause memory. Instead, use generator or iterator like **map or filter**

In [3]:
!pip install memory_profiler
%load_ext memory_profiler

In [4]:
def condition(datum):
    pass
def modify(datum):
    pass
dataset = [i for i in range(10000000)]

# Imperative
collection = []
%memit
for datum in dataset:
    if condition(datum):
        collection.append(datum)
    else:
        new = modify(datum)
        collection.append(new)

# Comprehension
%memit
collection = [datum if condition(datum) else modify(datum) for datum in dataset]

peak memory: 462.77 MiB, increment: 0.34 MiB
peak memory: 546.25 MiB, increment: 0.00 MiB


### Generators

- The generator comprehension has same syntax as list comprehension except square brackets, using parenthesis
- Use lazy-evaluation, so how to get datum is hiden until it is needed, just describe what data is
- **.next()** or **next looping** are used to access datum

### Iterator

```
iterCon = iter(iterable)
nextVal = next(iterCon)
```
- **itertools** module is used to control iterator
- Generator and Iterator is same internal logic, but make different ways
  - Generator is useful to make user-defined dataset
  - Iterator is useful to transform extra-defined dataset

In [5]:
def read_line(hugLogFile):
    pass
def complex_condition(line):
    pass

hugLogFile = [i for i in range(10)]

# Literal
#logLines = (line for line in read_line(hugLogFile) if complex_condition(line))

# Imperative
def get_log_lines(logFile):
    line = read_line(hugLogFile)
    while True:
        try:
            if complex_condition(line):
                yield line # Generator raise StopIteration when return statement occur(implicit or explicit)
            line = read_line(hugLogFile)
        except StopIteration:
            raise # same exception is passed

#logLines = get_log_lines(hugLogFile)

# Use Iterator protocol
class GetLogLines(object):
    def __init__(self, logFile):
        self.__logFile = logFile
        self.__line = None
    def __iter__(self):
        return self
    def __next__(self):
        if self.__line == None:
            self.__line = read_line(self.__logFile)
        while not complex_condition(self.__line):
            self.__line = read_line(self.__logFile)
        return self.__line

#logLines = getLogLines(hugLogFile)

# To compact dataset to memory size, use iterator
dataset = [i for i in range(100000)]
batch = next(iter(dataset))

In [6]:
# file I/O
x = open(r'./모든 컴퓨터 과학자가 알아야 할 부동 소수점의 모든것.pdf')
print('__iter__' in dir(x))
print('__next__' in dir(x))

#pandas I/O is not use iterable, so faster but, consum memory
#When changing iterator, the reference counts are decreased without current accessed datum. So, gc can clean memory
import sys
import seaborn as sns

%memit
iris = sns.load_dataset('iris')
print(sys.getrefcount(iris)) # To check current datum reference count
iris = iter(iris)
next(iris)

True
True
peak memory: 262.02 MiB, increment: -0.09 MiB
2


'sepal_length'

In [7]:
# map and filter make iterator
x = map(lambda x: x, range(100))
print('__iter__' in dir(x) and '__next__' in dir(x))
x = filter(lambda x: x%2 == 0, range(100)) # Passed function of filter must be the predicate function, return T/F
print('__iter__' in dir(x) and '__next__' in dir(x))

True
True


### Dics and Sets

#### Dics
```
{key: val for key, val in zip(keys, values)}
```

#### Sets
```
{mem for mem in data}
```

### Recursion

- In functional programming, the recursion expression replaces the loop statement
- In recursion style, Must distingush two cases
  - Just iteration by another name
  - A program can be readily be partitioned into smaller problems
- Effective recursion of pure functional language(not python)
  - Tail call elimination can be used to save memory
- In python, can be change recursion depth by ```sys.setrecursionlimit(cnt)```, default = 1000
- **functools** and **operator** is used to simplify recursion

In [8]:
# Just Iteration by another name
def sum(data):
    return data.pop() + sum(data) if data else 0

# Divide & Cunquer
def search(datum:"Datum", searchTree:"SearchTree")->"Node":
    if searchTree.root.datum == datum:
        return searchTree.root
    elif searchTree.root.datum < datum:
        return search(datum, searchTree.leftTree)
    else:
        return search(datum, searchTree.rightTree)

In [9]:
from functools import reduce
from operator import mul

def factorialHOF(n):
    return reduce(mul, range(1, n+1), 1)

### Eliminating Loop
- If function is called in loop simply, just use **map**

#### Translate While
- Statement based while loop
```
while <cond>:
    <pre-suite>
    if <break-condition>:
        break
    else:
        <suite>
```
- FP based while recursion
```
def while_block():
    <pre-suite>
    if <break-condition>:
        return 1
    else:
        <suite>
        return 0
        
whileFP = lambda: (<cond> and while_block()) or whileFP()
whileFP()
```
- It is not pure functional since ```while_block``` has statements

In [10]:
dataset = [i for i in range(pow(10,5))]

In [11]:
def f(i):
    i*i

%time
for datum in dataset:
    f(datum)

%time
map(f, dataset)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.62 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.1 µs


In [12]:
#map with passing arguments
do_it = lambda f, *args: f(*args)
fs = [lambda x, y: print(x, y), lambda x, y: print(x*x, y*y), lambda x, y: print(x*x*x, y*y*y)]
list(map(do_it, fs, [1,2,3], [4,5,6])) #map(func, *iterable, firstArguments, secondArguments, so on)

1 4
4 25
27 216


[None, None, None]

In [13]:
# Concrete version of whileFP
# Only expression, No side effects

#imperative
def echo_IMP():
    while True:
        x = input("IMP -- ")
        if x == "quit":
            break
        else:
            print(x)

from operator import eq
#FP
def identity_print(x):
    print(x)
    return x

echo_FP = lambda: eq(identity_print(input("IMP -- ")),"quit") or echo_FP()
echo_FP()

IMP -- quit
quit


True

### Eliminating recursion
- Sometimes recursion do without recursion by using folding operator like functools.reduce

#### Reduce
- Can be divide-Conquar
- In ML, almost operator adopt reduce-mechanism

#### Accumulate
- Can save memory and partial calculation
- But, slower than reduce

In [14]:
from operator import add
from functools import reduce
reduce(add, [1,2,3], 0)

# Accumulate is used for reducing iterable
from itertools import accumulate
x = accumulate([1,2,3], add)
print(next(x))
print(next(x))
print(next(x))

#numpy examples
import numpy as np
print(np.add.reduce([1,2,3,4,5]))
print(np.add.accumulate([1,2,3,4,5]))

1
3
6
15
[ 1  3  6 10 15]


## Callable

### Python Callable List
- Regular functions created by **def** keywords and given a name at definition time
- Anonymous functions created by **lambda** keywords
- Instances of classes that define a **__call__** method
- Closures returned by function factories
- Static methods of instances, either via the **@staticmethod** decorator or via the class **__dict__**
- Generator functions

### Non-static and Non-class methods of Class
- Usually used to access and modify members
- So, There are opposite to the functional paradigm, which emphisizes immutability and pure function
- There are used to implement OOP style

### Named functions and Lambdas
- The only in principle difference between them is simply whether its **\_\_qualname\_\_** is function name or not
- lambda is usually used when inlining or short expression
- lambda is **erased** when reference count is zero, but named functions remain over time

In [15]:
def f():
    pass
g = lambda x: x
print(f.__qualname__)
print(g.__qualname__)

f
<lambda>


### Closures and Callable instances
- Class is data with operations attached while Closure is operations with data attached
  - Class emphasizes mutable or rebindable states
  - Closure emphasizes immutability and pure functions
- Closure binds name rather than value
  - Scope manner is followed

In [16]:
# Callable Instances
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, m):
        return self.n + m

adderI = Adder(5)

# Closure
def make_adder(n):
    def adder(m):
        return n + m
    return adder

adderF = make_adder(5)

# Callable Instances have dependency to prior flows
print(adderI(5))
adderI.n = 10
print(adderI(5))

# Closure is pure
print(adderF(5))
adderF.n = 10 # Add member n, but it is different with adder's n
print(adderF.n)

# Bind name examples
adders = []
for n in range(1, 5):
    adders.append(lambda m: m+n)
print([adder(10) for adder in adders]) # bound n is current 4

adders = []
for n in range(1, 5):
    adders.append(lambda m, n=n: m+n) # explicit binding n=n
print([adder(10) for adder in adders])

def make_adder(n):
    def adder(m):
        return n + m
    n = 30 # binding n is here
    return adder

adderF = make_adder(10)
print(adderF(5) == 35)

10
15
10
10
[14, 14, 14, 14]
[11, 12, 13, 14]
True


### Method of Classes
- Calling method usually means modifying members

#### Accessors and Operators
- getter uses **@property** decorator
- setter uses **@member.setter** decorator
- In accessors, can co-opt the python syntax of assignment to pass arguments instead

In [17]:
class Car:
    def __init__(self):
        self.__speed = 80
    @property
    def speed(self):
        print("Car's speed is ", end='')
        return self.__speed
    
    @speed.setter
    def speed(self, new):
        print("Modified speed is", new)
        self.__speed = new
        
car = Car()
print(car.speed)
car.speed = 100
print(car.speed)

#Co-Opt python syntax
class TalkativeInt(int):
    def __lshift__(self, other):
        print('origin', self, 'by', other)
        return int.__lshift__(self, other)

x = TalkativeInt(1)
print(x << 3)

Car's speed is 80
Modified speed is 100
Car's speed is 100
origin 1 by 3
8


#### Static methods of Instances
- Use class just like namespace
- More functional style
- In python 3.x, **@staticmethod** decorator isn't needed, but if accessing static method in an instance, **Must** specify the decorator 

In [18]:
import math
class RightTriangle:
    '''Class used solely as namespace for related functions'''
    def hypotenuse(a:int, b:int) -> int:
        return math.sqrt(a**2+b**2)
    def sin(a:int, b:int) -> float:
        return a / RightTriangle.hypotenuse(a, b)
    def cos(a:int, b:int) -> float:
        return b / RightTriangle.hypotenuse(a, b)
    def tan(a:int, b:int) -> float:
        return a / b
    
print(RightTriangle.hypotenuse(3, 4))
print(RightTriangle.cos(3, 4))
print(RightTriangle.sin(3, 4))
print(RightTriangle.tan(3, 4))

rt = RightTriangle()
rt.cos(3, 4) # Error since self isn't passed

5.0
0.8
0.6
0.75


TypeError: RightTriangle.cos() takes 2 positional arguments but 3 were given

### Generator Functions
- function that contains **yield** statements
- Calling **next**, produce the sequence of values
- It use memory efficiently 'cause **lazy-evalutaion**
- Generator seems not to pure function since every calling **next()** may be different
- But, because First input affects the entire sequence of outputs, it is side-effect-free **blackbox** and pure function in a large category

In [19]:
def get_primes():
    '''Simple lazy Sieve of Erasthenes'''
    candidate = 2
    primes = []
    while True:
        if all(candidate%prime for prime in primes):
            yield candidate
            primes.append(candidate)
        else:
            candidate += 1

primes = get_primes()
for _ in range(10):
    print(next(primes), end = ' ')
for _, prime in zip(range(10), primes):
    print(prime, end = ' ')

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 

### Multiple Dispatch/Multi-Method
- Multiple Dispatch is the technique that is programming multiple path of executions

#### Idea
- Declare multiple signitures for a single function and call the actual computation that matches types or properties of calling parameters

#### Benefits
- Avoid explicit condition branches
- Instead substitute the use of more intuitive pattern descriptions of arguments

In [20]:
class Thing(object):
    ...
class Rock(Thing):
    ...
class Paper(Thing):
    ...
class Scissors(Thing):
    ...

# Imperative version
def beats(x, y):
    if not isinstance(x, Thing) or not isinstance(y, Thing):
        raise TypeError
    if isinstance(x, Rock):
        if isinstance(y, Rock):
            print("x(Rock), y(Rock): Tie")
        elif isinstance(y, Paper):
            print("x(Rock), y(Paper): y win")
        elif isinstance(y, Scissors):
            print("x(Rock), y(Scissors): x win")
        else:
            raise TypeError
    elif isinstance(x, Paper):
        if isinstance(y, Rock):
            print("x(Paper), y(Rock): x win")
        elif isinstance(y, Paper):
            print("x(Paper), y(Paper): tie")
        elif isinstance(y, Scissors):
            print("x(Paper), y(Scissors): y win")
        else:
            raise TypeError  
    elif isinstance(x, Scissors):
        if isinstance(y, Rock):
            print("x(Scissors), y(Rock): y win")
        elif isinstance(y, Paper):
            print("x(Scissors), y(Paper): x win")
        elif isinstance(y, Scissors):
            print("x(Scissors), y(Scissors): tie")
        else:
            raise TypeError   
    else:
        raise TypeError
    return

cmb = Rock(), Paper(), Scissors()
for i in range(3):
    for j in range(3):
        beats(cmb[i], cmb[j])


x(Rock), y(Rock): Tie
x(Rock), y(Paper): y win
x(Rock), y(Scissors): x win
x(Paper), y(Rock): x win
x(Paper), y(Paper): tie
x(Paper), y(Scissors): y win
x(Scissors), y(Rock): y win
x(Scissors), y(Paper): x win
x(Scissors), y(Scissors): tie


#### Delgate to the Objects
- With **duck-typing**, eleminate branches
- Actually, by nesting branches in each classes, reduce complexity code and the level of nested
- But, the amount of code isn't reduced
- In OOP, can use delegate patterns to delate dispatch to the object

In [21]:
class DuckRock(Rock):
    def beat(self, other):
        if isinstance(other, Rock):
            print("x(Rock), y(Rock): Tie")
        elif isinstance(other, Paper):
            print("x(Rock), y(Paper): y win")
        elif isinstance(other, Scissors):
            print("x(Rock), y(Scissors): x win")
        else:
            raise TypeError
        
class DuckPaper(Paper):
    def beat(self, other):
        if isinstance(other, Rock):
            print("x(Paper), y(Rock): x win")
        elif isinstance(other, Paper):
            print("x(Paper), y(Paper): tie")
        elif isinstance(other, Scissors):
            print("x(Paper), y(Scissors): y win")
        else:
            raise TypeError         
    
class DuckScissors(Scissors):
    def beat(self, other):
        if isinstance(other, Rock):
            print("x(Scissors), y(Rock): y win")
        elif isinstance(other, Paper):
            print("x(Scissors), y(Paper): x win")
        elif isinstance(other, Scissors):
            print("x(Scissors), y(Scissors): tie")
        else:
            raise TypeError 

# Delegate version
def beats2(x, y):
    if hasattr(x, 'beat'):
        return x.beat(y)
    else:
        raise TypeError
    return

duckCmb = DuckRock(), DuckPaper(), DuckScissors()
for i in range(3):
    for j in range(3):
        beats2(duckCmb[i], duckCmb[j])


x(Rock), y(Rock): Tie
x(Rock), y(Paper): y win
x(Rock), y(Scissors): x win
x(Paper), y(Rock): x win
x(Paper), y(Paper): tie
x(Paper), y(Scissors): y win
x(Scissors), y(Rock): y win
x(Scissors), y(Paper): x win
x(Scissors), y(Scissors): tie


#### Pattern-Matching
- Using multiple dispatches, can express branches more directly
- this should be more readable, though still a number of case defined

In [ ]:
!pip install multipledispatch

In [22]:
from multipledispatch import dispatch

@dispatch(Rock, Rock)
def beats3(x, y): print('tie')
@dispatch(Rock, Paper)
def beats3(x, y): print('y win')
@dispatch(Rock, Scissors)
def beats3(x, y): print('x win')
@dispatch(Paper, Rock)
def beats3(x, y): print('x win')
@dispatch(Paper, Paper)
def beats3(x, y): print('tie')
@dispatch(Paper, Scissors)
def beats3(x, y): print('y win')
@dispatch(Scissors, Rock)
def beats3(x, y): print('y win')
@dispatch(Scissors, Paper)
def beats3(x, y): print('x win')
@dispatch(Scissors, Scissors)
def beats3(x, y): print('tie')
@dispatch(object, object)
def beats3(x, y):
    raise TypeError


cmb = Rock(), Paper(), Scissors()
for i in range(3):
    for j in range(3):
        beats3(cmb[i], cmb[j])

tie
y win
x win
x win
tie
y win
y win
x win
tie


#### Predicate-based Dispatch
- A real exotic approach to expressing conditionals as dispatch decisions is to include predicates directly within the function signatures
- Or perhaps within decorators on them, as with multipledispatch
- No eligble library in python. So, illustrate imaginary library
- Predicate is placed outside function, thus the depth of condition is shallower and more readable

In [23]:
from predicate_dispatch import predicate

@predicate(lambda x: x < 0, lambda y: True)
def sign(x, y):
    print("x is negative, y =", y)
@predicate(lambda x: x == 0, lambda y: True)
def sign(x, y):
    print("x is zero, y =", y)
@predicate(lambda x: x > 0, lambda y: True)
def sign(x, y):
    print("x is positive, y =", y)
    

ModuleNotFoundError: No module named 'predicate_dispatch'

## Lazy Evaluation

- Python doesn't quite offer **lazy data structures** in the sense of Haskell. But, with **iterator protocol** and builtin or standard libraries for **iterable**, it can be accomplished
  - Haskell's lazy data structure example
  ```
  primes = sieve [2 ..]
    where sieve (p:xs) = p :sieve [x|x <- xs, (x `rem` p) /= 0]
  ```
  - Python's lazy data structure example 

In [24]:
# Lazy and Indexible data structure

from collections.abc import Sequence
from operator import eq, le
class ExpandingSequence(Sequence):
    def __init__(self, it):
        self.__it = it
        self.__cache = []
    def __len__(self):
        return len(self.__cache)
    def __getitem__(self, idx):
        lazyEval = lambda: self.__cache.append(next(self.__it))
        lazySeq = lambda: le(len(self.__cache), idx) and eq(lazyEval(), None) and lazySeq()
        lazySeq()
        return self.__cache[idx]

primes = ExpandingSequence(get_primes())
print(len(primes))
print(primes[100])
print(len(primes))

0
547
101


### Iterator protocol
- The easiest way to **create** an iterator, that is, a lazy sequence, in Python is to define **generator** 
- The easiest way to **use** an iterator, is to use one of many iterable objects already produced by builtin or standard libraries
- By duck-typing, iterable objects must have **\_\_iter\_\_** method, which return **iterator**
  - To make the iterator of the iterable object, simply use **iter(iterable)** builtin function. It return iterable.\_\_iter\_\_()
- With **\_\_next\_\_()** method, **\_\_iter\_\_()** just returns itself and calcuate the next by **\_\_next\_\_** method

#### Abstract class
- Iterable is the protocol to make iterable structures
  - \_\_iter\_\_(self) is the only interface to fulfill iterable
  - To convert it to iterator, \_\_iter\_\_ **must** return **the iterator**
  - **iter(iterable) returns iterable.\_\_iter\_\_(self)**
- Iterator is the protocol to make iterator structures
  - Iterator is a subclass of Iterable
  - \_\_next\_\_(self) is the interface to fulfill iterator
  - **next(iterator) returns iterator.\_\_next\_\_(self)**
  
#### Idempotent of iterators
- An iterator is the iterator. So, ```iter(iterator) == iterator``` and ```iter(iterator) is iterator```

In [25]:
# Use iterator protocol
from collections.abc import Iterator, Iterable

print(issubclass(Iterator, Iterable))

class Fibonacci(Iterator):
    def __init__(self):
        self.__a, self.__b = 0, 1
        self.__total = 0
    def __iter__(self):
        return self
    def __next__(self):
        self.__a, self.__b = self.__b, self.__a + self.__b
        self.__total += self.__a
        return self.__a
    def get_running_sum(self):
        return self.__total

fibo = Fibonacci()
for i in range(10):
    print("fibo{idx}: {val}, runing sum: {sum}".format(idx=i, val=next(fibo), sum=fibo.get_running_sum()))


True
fibo0: 1, runing sum: 1
fibo1: 1, runing sum: 2
fibo2: 2, runing sum: 4
fibo3: 3, runing sum: 7
fibo4: 5, runing sum: 12
fibo5: 8, runing sum: 20
fibo6: 13, runing sum: 33
fibo7: 21, runing sum: 54
fibo8: 34, runing sum: 88
fibo9: 55, runing sum: 143


### Module: Itertools
- A collection of very useful functions for performing iterator algebra
- **more_itertools** is also an useful third-party librariy

#### Goal
- Avoid evalutaions it is really required. So, reduce memory and very slow I/O oepration

In [ ]:
!pip install more_itertools

In [26]:
# Simplify Above Fibonacci Class

from itertools import tee, accumulate

def fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b

s, t = tee(fibonacci()) # tee split an iteator as two independent same iterators
fibo = zip(s, accumulate(t)) # accumulate do partial summing
for _, (x, y) in zip(range(10), fibo):
    print(x, y)

1 1
1 2
2 4
3 7
5 12
8 20
13 33
21 54
34 88
55 143


#### Chaning Iteratables

- ```itertools.chain()``` or ```itertools.chain.from_iterable()``` functions combine multiple iterables
  - **chain(\*iterable)** concatenates the iterables in the manner of end to begin
  - **chain.from_iterable(iterable)** make chain object with next(iterable) as its arguments
  
- ```zip()``` or ```itertools.zip_longest()``` functions also do this

In [27]:
from itertools import chain, zip_longest

x = chain([1], [2])
for _ in range(2):
    print(next(x))

def f():
    def g():
        x = 1
        while x < 3:
            yield x
            x += 1
    while True:
        yield g()
        print('next g()')

x = chain.from_iterable(f())
for _ in range(10):
    print(next(x))

for x, y in zip_longest([1], [1,2]): # shorter iterable fill its place default value, such as None
    print(x, y)

1
2
1
2
next g()
1
2
next g()
1
2
next g()
1
2
next g()
1
2
1 1
None 2


#### Chaining Other structures
- ```collections.ChainMap``` concatenates maps, generally subclass of ```collections.abs.Mapping``` like Dictionary
- It also iterable over their keys
- Without creating Single larger Map, just concatenate maps

In [28]:
from collections import ChainMap

x = ChainMap({1: 1}, {2:2})

## Higher-order functions(HOFs)
- Like iterable algebra, HOFs provide similar building blocks to express the complex concept
- In general, HOF means that the function takes functions and produce a function
- ```functools``` has useful HOFs
  - ```reduce``` is an useful HOF
- ```map```, ```filter``` are also useful builtin HOFs

### Basic HOFs
- ```map```, ```filter```, ```functools.reduce``` are basic building blocks
  - ```map``` and ```filter``` are equivalent to comprehensions
  - ```functools.reduce``` can produce other HOFs
    - Anything that can be computed from a sequence of successive elements can be expressed as **reduce**
- ```functools.partial``` is also useful. It makes **currying**

In [29]:
from functools import reduce, partial

# Classic FP-style
transformed = map(lambda x: x + 1, [1,2,3])

# Comprehension
def transform(x): return x + 1
transformed = (transform(x) for x in [1,2,3])

# Classic FP-style
filtered = filter(lambda x: x%2 == 0, [1,2,3,4])

# Comprehension
def predicate(x): return x%2 == 0
filtered = (x for x in [1,2,3,4] if predicate(x))

# Make map by reduce
def add5(x): return x + 5
map_by_reduce = lambda f, lst: iter(reduce(lambda l, x: l + [f(x)], lst, []))

# Make filter by reduce
def predicate(x): return x % 2 == 0
filter_by_reduce = lambda f, lst: iter(reduce(lambda l, x: l + [x] if predicate(x) else l, lst, []))

# partial
def f(a, b, c, d, e):
    print(a, b, c, d,e)
partialF = partial(f, 1, 2, 3)
partialF(4,5)

1 2 3 4 5


### The operator Module
- Every operations that can be done with Python's infix and prefix operators corresponds to a named function in **operator** module

In [ ]:
from operator import add
add(3,4)

### The functools Module
- Python's HOFs is placed in **functools** obviously

### Decorator
- The most common use of HOFs is as decorators
- A decorator is just a **syntax sugar** that takes a function as an argument and produce the enhanced function as result
- In some ways, decorator tie in **Aspect Oriented Programming**
- Decorators in general are more useful when you want to **poke into the guts of a function** than when you want to treat it as a pluggable component in a flow or composition of functions, often done to mark the purpose or capabilities of a particular function.

#### Functools's decorators
- ```functools.lru_cache```
  - memorize function's arguments and result pair and return stored value rather than re-computation
  - To use it, decorated function **must be pure**
- ```functools.total_ordering```
  - make it easier to write costum classes that want to use inequality operators
  - If implementing ```\_\_eq\_\_``` and one of ```\_\_gt\_\_```, ```\_\_ge\_\_```, ```\_\_lt\_\_```, ```\_\_le\_\_```, then remains are written automatically
  - But, its speed is slower than explicit implemantations
- ```functools.wraps```
  - make it easier to write custom decorators

In [30]:
def enhanced(f):
    print("enhanced")
    return f

@enhanced
def something(x):
    print(x)

enhanced


In [31]:
from functools import lru_cache, total_ordering, wraps

x = 1
@lru_cache
def f():
    global x
    print('calculated')
    return x*x
# print(f())
# x = 2
# print(f()) # Unexpected result 'cause f isn't pure

@lru_cache
def f(x):
    print('calculated')
    return x*x
# print(f(1))
# print(f(1))

@total_ordering
class customInt:
    def __init__(self, n):
        self.n = n
    def __eq__(self, other):
        if isinstance(other, A):
            return self.n == other.n
        return False
    def __lt__(self, other):
        if isinstance(other, A):
            return self.n < other.n
        return False

x = customInt(1)
y = customInt(2)
# print(x < y)
# print(x > y)
# print(x <= y)
# print(x >= y)
# print(x == y)

def custom_decorator(f):
    @wraps(f)
    def wrapper(*args:"cool", **kwds:"happy")->"join":
        '''wrapper docstring'''
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@custom_decorator
def example():
    '''Docstring'''
    print('Called example function')
example()
# print(example.__name__) #without wraps, name is wrapper
# print(example.__doc__) #without wraps, doc is wrapper's docstring
# example #witout wraps, signiture's annotation is wrapper's annotation

Calling decorated function
Called example function
